In [25]:
import pandas as pd
from pandas.io.json import json_normalize
import json, requests
import config

In [26]:
endpoint = 'https://ridb.recreation.gov/api/v1/organizations/{orgID}/recareas'
org_id = 128

In [68]:
offset=0
params = dict(apiKey= config.RIDB_API_KEY, offset=offset)
nps_url = endpoint.replace('{orgID}', str(org_id))
resp = requests.get(url=nps_url, params=params)
data = json.loads(resp.text)
df = json_normalize(data['RECDATA'])
df_nps = df


In [69]:
max_records = data['METADATA']['RESULTS']['TOTAL_COUNT']

In [70]:
df_nps.shape

(50, 16)

In [71]:
while offset < max_records:
    offset = offset + len(df)
    print("offset: " + str(offset))
    df = pd.DataFrame()
    params = dict(apiKey= config.RIDB_API_KEY, offset=offset)
    try :
        resp = requests.get(url=nps_url, params=params)
    except Exception as ex:
        print(ex)
        break
    if resp.status_code == 200:
        data = json.loads(resp.text)
        if data['METADATA']['RESULTS']['CURRENT_COUNT'] > 0 :
            df = json_normalize(data['RECDATA'])
            df_nps = df_nps.append(df)
    else :
        print ("Response: " + str(resp.status_code))

offset: 50
offset: 100
offset: 150
offset: 200
offset: 250
offset: 300
offset: 350
offset: 400
offset: 450
offset: 500
offset: 550
offset: 590


In [72]:
df_nps.shape

(590, 18)

In [82]:
df_np = df_nps[df_nps['RecAreaName'].apply(lambda x: x.find('National Park') > 0)]

In [115]:
df_np.shape

(61, 18)

In [97]:
df_np[df_np['RecAreaLongitude'] == ""].RecAreaName

21                            Arches National Park
38      Black Canyon Of The Gunnison National Park
18                  Carlsbad Caverns National Park
47                 Denali National Park & Preserve
43    Gates Of The Arctic National Park & Preserve
49                           Glacier National Park
16               Guadalupe Mountains National Park
21                         Haleakalā National Park
43                 Katmai National Park & Preserve
9                    Lassen Volcanic National Park
21               Western Arctic National Parklands
9                Congaree National Park Wilderness
48         Rocky Mountain National Park Wilderness
Name: RecAreaName, dtype: object

In [108]:
df_np[df_np['RecAreaName'] == 'Rocky Mountain National Park']

,GEOJSON,GEOJSON.COORDINATES,GEOJSON.TYPE,Keywords,LastUpdatedDate,OrgRecAreaID,RecAreaDescription,RecAreaDirections,RecAreaEmail,RecAreaFeeDescription,RecAreaID,RecAreaLatitude,RecAreaLongitude,RecAreaMapURL,RecAreaName,RecAreaPhone,RecAreaReservationURL,StayLimit
49,NaN,"[-105.562038422, 40.3663213198]",Point,,2016-10-11,,Rocky Mountain National Park's 415 square mile...,For directions to the park visit http://www.np...,romo_information@nps.gov,,2907,40.3663,-105.562,http://www.nps.gov/romo/planyourvisit/maps.htm,Rocky Mountain National Park,(970) 586 1206,,


In [99]:
df_np[['RecAreaLatitude', 'RecAreaLongitude']].head()

,RecAreaLatitude,RecAreaLongitude
3,44.3744,-68.2599
21,,
25,43.7489,-101.942
30,29.329,-103.207
34,25.483,-80.2179


In [150]:
missing_latlongs = pd.read_csv('missing_lat_longs.csv')
missing_latlongs.head()

,RecAreaName,RecAreaLatitude,RecAreaLongitude
0,Arches National Park,38.7331,-109.5925
1,Black Canyon Of The Gunnison National Park,38.5754,-107.7416
2,Carlsbad Caverns National Park,32.1479,-104.5567
3,Denali National Park & Preserve,63.1148,-151.1926
4,Gates Of The Arctic National Park & Preserve,67.9204,-153.2753


In [122]:
missing_latlongs = missing_latlongs.set_index('RecAreaName')
df_np = df_np.set_index('RecAreaName')

In [123]:
df_np.update(missing_latlongs)

In [126]:
df_np[df_np['RecAreaLongitude'] == ""].index

Index(['Western Arctic National Parklands',
       'Congaree National Park Wilderness',
       'Rocky Mountain National Park Wilderness'],
      dtype='object', name='RecAreaName')

In [127]:
df_np = df_np[df_np['RecAreaLongitude'] != ""]

In [129]:
df_np.shape

(58, 17)

In [130]:
df_np['RecAreaName'] = df_np.index

In [131]:
df_np.shape

(58, 18)

In [136]:
df_np['newIndex'] = range(0,58)

In [137]:
df_np.set_index(df_np['newIndex'])

,GEOJSON,GEOJSON.COORDINATES,GEOJSON.TYPE,Keywords,LastUpdatedDate,OrgRecAreaID,RecAreaDescription,RecAreaDirections,RecAreaEmail,RecAreaFeeDescription,RecAreaID,RecAreaLatitude,RecAreaLongitude,RecAreaMapURL,RecAreaPhone,RecAreaReservationURL,StayLimit,RecAreaName,newIndex
newIndex,,,,,,,,,,,,,,,,,,,
0,"{'TYPE': 'Point', 'COORDINATES': [-68.25986579...",NaN,NaN,,2016-10-11,,People have been drawn to the rugged coast of ...,"From Boston take I-95 north to Augusta, Maine,...",acadia_information@nps.gov,,2554,44.3744,-68.2599,http://www.nps.gov/acad/planyourvisit/maps.htm,(207) 288 3338,,,Acadia National Park,0
1,NaN,NaN,NaN,,2016-10-11,,Visit Arches and discover a landscape of contr...,Arches National Park is located in southeast U...,archinfo@nps.gov,,2573,38.7331,-109.593,http://www.nps.gov,,,,Arches National Park,1
2,"{'TYPE': 'Point', 'COORDINATES': [-101.9422077...",NaN,NaN,,2016-10-11,,The rugged beauty of the Badlands draws visito...,,badl_information@nps.gov,,2578,43.7489,-101.942,http://www.nps.gov/badl/planyourvisit/maps.htm,,,,Badlands National Park,2
3,"{'TYPE': 'Point', 'COORDINATES': [-103.2072454...",NaN,NaN,,2016-10-11,,There is a place in Far West Texas where night...,Several Texas highways lead to Big Bend Nation...,bibe_info@nps.gov,,2584,29.329,-103.207,http://www.nps.gov/bibe/planyourvisit/maps.htm,(432) 477 2251,,,Big Bend National Park,3
4,"{'TYPE': 'Point', 'COORDINATES': [-80.21793365...",NaN,NaN,,2016-10-11,,"Within sight of downtown Miami, yet worlds awa...",<p>Physical Address<br />\n9700 SW 328 Street<...,bisc_information@nps.gov,,2588,25.483,-80.2179,http://www.nps.gov/bisc/planyourvisit/maps.htm,,,,Biscayne National Park,4
5,NaN,NaN,NaN,,2016-10-11,,"Big enough to be overwhelming, still intimate ...",<p>Black Canyon of the Gunnison National Park ...,CURE_Info@nps.gov,,2592,38.5754,-107.742,http://www.nps.gov/blca/planyourvisit/maps.htm,(970) 641 2337,,,Black Canyon Of The Gunnison National Park,5
6,"{'TYPE': 'Point', 'COORDINATES': [-112.1699514...",NaN,NaN,,2016-10-11,,There is no place like Bryce Canyon. Hoodoos (...,http://www.nps.gov/brca/planyourvisit/directio...,brca_information@nps.gov,,2599,37.6392,-112.17,http://www.nps.gov/brca/planyourvisit/maps.htm,,,,Bryce Canyon National Park,6
7,NaN,"[-109.856624603, 38.3121026863]",Point,,2016-10-11,,Canyonlands invites you to explore a wildernes...,Car\nThere are two paved entrances into Canyon...,canyinfo@nps.gov,,2616,38.3121,-109.857,http://www.nps.gov/cany/planyourvisit/maps.htm,,,,Canyonlands National Park,7
8,NaN,"[-111.261219024, 38.2906477893]",Point,,2016-10-11,,Located in south-central Utah in the heart of ...,<p>Directions to Capitol Reef National Park</p...,CARE_Interpretation@nps.gov,,2617,38.2906,-111.261,http://www.nps.gov,(435) 425 3791,,,Capitol Reef National Park,8


In [146]:
df_np = df_np.drop('newIndex', axis=1)

In [147]:
df_np.shape

(58, 18)

In [148]:
df_np.columns

Index(['GEOJSON', 'GEOJSON.COORDINATES', 'GEOJSON.TYPE', 'Keywords',
       'LastUpdatedDate', 'OrgRecAreaID', 'RecAreaDescription',
       'RecAreaDirections', 'RecAreaEmail', 'RecAreaFeeDescription',
       'RecAreaID', 'RecAreaLatitude', 'RecAreaLongitude', 'RecAreaMapURL',
       'RecAreaPhone', 'RecAreaReservationURL', 'StayLimit', 'RecAreaName'],
      dtype='object')

In [149]:
df_np.to_csv('np_info.csv')

Convert the df_np data into geojson format for use by google maps

In [151]:
collection = {'type':'FeatureCollection', 'features':[]}

In [153]:
def feature_from_row(title, latitude, longitude):
    feature = { 'type': 'Feature', 
               'properties': { 'title': ''},
               'geometry': { 'type': 'Point', 'coordinates': []}
               }
    feature['geometry']['coordinates'] = [longitude, latitude]
    feature['properties']['title'] = title
    collection['features'].append(feature)
    return feature

In [ ]:

geojson_series = df_np.apply(lambda x: feature_from_row(x['title'],x['latitude'],x['longitude'],x['description']),
                                  axis=1)